# Simple Classification

## 1. Setup

In [ ]:
import torch
from torch import nn, optim
import random
import math
import numpy as np
from matplotlib import pyplot as plt
from IPython import display

In [ ]:
from common.plot_tools import plot_data, plot_model, set_default

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
torch.manual_seed(12345)

In [ ]:
n_samples_per_class = 1000
n_classes = 4
n_dims = 2

In [ ]:
set_default()

## 2. Creating an artificial dataset

In [ ]:
X = torch.zeros(n_samples_per_class * n_classes, n_dims).to(device)
y = torch.zeros(n_samples_per_class * n_classes, dtype=torch.long).to(device)

for c in range(n_classes):
    index = 0
    t = torch.linspace(0, 1, n_samples_per_class)
    inner_var = torch.linspace(
        (2 * math.pi / n_classes) * (c),
        (2 * math.pi / n_classes) * (2 + c),
        n_samples_per_class
    ) + torch.randn(n_samples_per_class) * 0.2
    
    for ix in range(n_samples_per_class * c, n_samples_per_class * (c + 1)):
        X[ix] = t[index] * torch.FloatTensor((
            math.sin(inner_var[index]), math.cos(inner_var[index])
        ))
        y[ix] = c
        index += 1

print("Shapes:")
print("X:", tuple(X.size()))
print("y:", tuple(y.size()))

In [ ]:
# visualise the data
plot_data(X, y)

## 3. Linear Model

In [ ]:
learning_rate = 1e-3
lambda_l2 = 1e-5
n_hidden = 100

In [ ]:
# create the model
model = nn.Sequential(
  nn.Linear(n_dims, n_hidden),
  nn.Linear(n_hidden, n_classes)                     
)

# send it to the device
model.to(device)

# create the loss criterion
criterion = nn.CrossEntropyLoss()

# create the optimizer
optimizer = optim.SGD(model.parameters(), lr=learning_rate, weight_decay=lambda_l2)

In [ ]:
# training loop
for t in range(1000):
  
  # feed the input forward through the network
  y_pred = model(X)

  # compute th loss
  loss = criterion(y_pred, y)
  _, predicted = torch.max(y_pred, dim=1)
  acc = (y == predicted).sum().float() / len(y)
  print("[EPOCH]: %i, [LOSS]: %.6f, [ACCURACY]: %.3f" % (t, loss.item(), acc))
  display.clear_output(wait=True)

  # zero the gradients
  optimizer.zero_grad()

  # backward pass to compute the gradients
  loss.backward()

  # update params
  optimizer.step()

In [ ]:
# Plot trained model
print(model)
plot_model(X, y, model)

## 4. Non-linear Model

In [ ]:
learning_rate = 1e-3
lambda_l2 = 1e-5

In [ ]:
# create the model
model = nn.Sequential(
    nn.Linear(n_dims, n_hidden),
    nn.ReLU(),
    nn.Linear(n_hidden, n_classes)
)

# send to device
model.to(device)

# loss criterion
criterion = torch.nn.CrossEntropyLoss()

# define optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=lambda_l2) # built-in L2

In [ ]:
# training loop
for t in range(1000):
    
    # feed the input forward into the network
    y_pred = model(X)
    
    # Compute the loss and accuracy
    loss = criterion(y_pred, y)
    _, predicted = torch.max(y_pred, 1)
    acc = (y == predicted).sum().float() / len(y)
    print("[EPOCH]: %i, [LOSS]: %.6f, [ACCURACY]: %.3f" % (t, loss.item(), acc))
    display.clear_output(wait=True)
    
    # zero the grads
    optimizer.zero_grad()
    
    # compute the gradients 
    loss.backward()
    
    # update params
    optimizer.step()

In [ ]:
# Plot trained model
print(model)
plot_model(X, y, model)